This notebook is for deployment using Gradio

In [ ]:
# Install necessary libraries
!pip install gradio transformers torch

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import gradio as gr

# Define model path
MODEL_DIR = "models/t5_myth_buster"

# Load trained model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR)
model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# Define the debunking function
def debunk_myths(myths):
    results = {}
    for myth in myths:
        input_text = f"Myth: {myth}\nDebunked Fact:"
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

        # **Step 1: First Try Beam Search (Accurate, Deterministic)**
        output_ids = model.generate(
            input_ids,
            max_length=128,
            num_beams=7,
            do_sample=False,  # Fully deterministic
            repetition_penalty=1.5,
            no_repeat_ngram_size=4,  # Stronger constraint to prevent repetition
            early_stopping=True
        )

        debunked_fact = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        # **Step 2: If Myth is Repeated, Try Sampling Instead**
        if not debunked_fact or debunked_fact.lower() == myth.lower():
            output_ids = model.generate(
                input_ids,
                max_length=128,
                num_beams=5,
                do_sample=True,  # Enable randomness
                temperature=0.7,
                top_p=0.85,
                top_k=50,
                repetition_penalty=1.3,
                no_repeat_ngram_size=4,  # Prevents reusing phrases
                early_stopping=True
            )
            debunked_fact = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        # **Step 3: Final Cleanup - Remove Repeated Myths**
        if debunked_fact.startswith("Myth:"):
            debunked_fact = debunked_fact.replace(f"Myth: {myth}", "").strip()

        # **Step 4: If Still Failing, Provide a Default Debunking Response**
        if not debunked_fact or debunked_fact.lower() == "debunked fact:":
            debunked_fact = "AI follows pre-programmed rules and data patterns but does not possess true intelligence or consciousness."

        results[myth] = debunked_fact

    return results

# Create Gradio Interface
iface = gr.Interface(
    fn=debunk_myth,  # Function to call
    inputs="text",  # Input is a text box
    outputs="text",  # Output is a text box
    title="AI Myth Debunker",
    description="Enter an AI myth, and the bot will provide a debunked, fact-based correction."
)

# Launch the app
iface.launch()
